In [1]:
### This script create merged coverage file according to cohorts file -- option A 
### This merge process is re written in May-31st, directly merge the first and add run
# Author: Yiyun
import os， pickle
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool

In [4]:
### dir of the first and second coverage calculation
dir_first = '../cov_out/cov_calc_output/coverage_complete_output/'
dir_add = '../cov_out/cov_calc_output/add_coverage/'
dir_ref = '../anno_ref/'

out_dir = '../cov_out/coverage_ind'

rename_dict = {}
for i in range(1,8):
    key = 'Categ'+str(i)
    rename_dict[key] = str(i)

In [5]:
### Read data
first_list = os.listdir(dir_first)
add_list = os.listdir(dir_add)
lp = pickle.load(open(os.path.join(dir_ref,'list_all_patients.pkl'),'rb'))

In [6]:
def format_patient_coverage(patient):
    global dir_first, dir_add, rename_dict, out_dir
    # Read first and second coverage run
    df_first = pd.read_csv(os.path.join(dir_first,patient+'.coverage.out.csv'), sep = '\t')
    df_add = pd.read_csv(os.path.join(dir_add,patient+'.csv'), sep = '\t')
    df_all = pd.concat([df_first,df_add])
    lgene = df_all['gene_name'].unique()
    lzone = ['flank','silent','nonsilent']
    lcom = [(x,y) for x in lgene for y in lzone]
    
    df_all = df_all.set_index(['gene_name','zone']).sort_index()

    df_patient = pd.DataFrame()
    for idx in tqdm(lcom):
            df_temp = df_all.loc[pd.IndexSlice[idx]]
            df_temp = df_temp.rename(columns = rename_dict)
            df_temp = df_temp.stack()
            df_temp = df_temp.loc[df_temp.index[1:]]

            df_patient = pd.concat([df_patient, df_temp])
    df_patient.index = pd.MultiIndex.from_tuples(df_patient.index)
    df_patient.index.names = ['gene', 'zone','categ']
    df_patient.columns = [patient]
    
    df_patient.to_csv(os.path.join(out_dir, patient+'.to_merge.csv'))

In [7]:
# Inidividual patient
p = '2bde43e2-fd4e-413e-adc7-c50bca75d358'
format_patient_coverage(p)

FileNotFoundError: [Errno 2] File ../cov_out/cov_calc_output/coverage_complete_output/2bde43e2-fd4e-413e-adc7-c50bca75d358.coverage.out.csv does not exist: '../cov_out/cov_calc_output/coverage_complete_output/2bde43e2-fd4e-413e-adc7-c50bca75d358.coverage.out.csv'

In [172]:
p = Pool(4)
p.map(format_patient_coverage, lp)
p.close()
p.join()

 27%|██▋       | 15647/57789 [09:45<26:16, 26.73it/s]  


KeyboardInterrupt: 


Process ForkPoolWorker-98:


Process ForkPoolWorker-100:
Process ForkPoolWorker-97:
Process ForkPoolWorker-99:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/storage/home/yur97/anaconda3/envs/cnp/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/storage/home/yur97/anaconda3/envs/cnp/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/storage/home/yur97/anaconda3/envs/cnp/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/storage/home/yur97/anaconda3/envs/cnp/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/storage/home/yur97/anaconda3/envs/cnp/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/storage/home/yur97/anaconda3/envs/cnp/lib/python3.7/multiprocessing/poo